In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
import sys

# === Load the trained model
model = load_model('best_inception_model.keras')

# === Image preprocessing function
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found or invalid: {image_path}")
    img = cv2.resize(img, (150, 150))
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)
    return img

# === Diagnosis function
def diagnose(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)[0][0]  # Get single prediction
    if prediction > 0.5:
        result = "🧠 Tumor Detected"
    else:
        result = "✅ No Tumor Detected"
    print(f"Image: {image_path}")
    print(f"Prediction Confidence: {prediction:.4f}")
    print(f"Diagnosis Result: {result}")

# === Main
if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python diagnose_brain_tumor.py <path_to_image>")
    else:
        diagnose(sys.argv[1])
